In [7]:
import boto3
import pandas as pd
import idx2numpy
import numpy as np
from os.path  import join

import logging
from botocore.exceptions import ClientError
import os

In [2]:
s3 = boto3.client("s3")

In [3]:
x_train= "train-images.idx3-ubyte"
y_train= "train-labels.idx1-ubyte"
x_test= "t10k-images.idx3-ubyte"
y_test= "t10k-labels.idx1-ubyte"

s3.download_file(
    "training-data-sagemaker-tensorflow-mnist",
    x_train,
    "train-images.idx3-ubyte",
)
s3.download_file(
    "training-data-sagemaker-tensorflow-mnist",
    y_train,
    "train-labels-idx1-ubyte",
)
s3.download_file(
    "training-data-sagemaker-tensorflow-mnist",
    x_test,
    "t10k-images-idx3-ubyte",
)
s3.download_file(
    "training-data-sagemaker-tensorflow-mnist",
    y_test,
    "t10k-labels-idx1-ubyte",
)

In [4]:
input_path = ''

training_images_filepath = join(input_path, 'train-images.idx3-ubyte')
training_labels_filepath = join(input_path, 'train-labels-idx1-ubyte')
test_images_filepath = join(input_path, 't10k-images-idx3-ubyte')
test_labels_filepath = join(input_path, 't10k-labels-idx1-ubyte')

In [5]:
x_train = idx2numpy.convert_from_file(training_images_filepath)
y_train = idx2numpy.convert_from_file(training_labels_filepath)
x_test = idx2numpy.convert_from_file(test_images_filepath)
y_test = idx2numpy.convert_from_file(test_labels_filepath)

In [6]:
np.savez("mnist.npz", x_test=x_test, x_train=x_train, y_train=y_train, y_test=y_test)

In [8]:
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [10]:
upload_file("mnist.npz", "training-data-sagemaker-tensorflow-mnist","processing_output/mnist.npz")

True